# Data Filtering and Selection

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import zscore
from textblob import TextBlob
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append('../utils')
import functions

In [ ]:
movie_df = pd.read_csv('../data/local/clean/films_19to24.csv')
movie_df.head()

### Remove Rows with Irrelevant information
- Rows with missing or incomplete data
- Rows with outliers or extreme values
- Rows with zero or negative revenue and budget
- Rows with zero or extremely low ratings
- Duplicates
- Rows with low votes
- 

In [ ]:
df_cleaned = movie_df.copy()

In [ ]:
# # Step 1: Remove rows with critical missing values (imdb_rating, imdb_votes, imdb_id)
# initial_row_count = len(df)
# df_cleaned = df.dropna(subset=['imdb_rating', 'imdb_votes', 'imdb_id'])
# rows_removed_step_1 = initial_row_count - len(df_cleaned)
# print(f'Rows removed after dropping rows with missing critical values: {rows_removed_step_1}')

In [ ]:
# # Step 2: Remove rows with extreme z-scores for budget, revenue, or popularity
# initial_row_count = len(df_cleaned)
# df_cleaned = df_cleaned[(df_cleaned['budget_zscore'].abs() < 3) & 
#                         (df_cleaned['revenue_zscore'].abs() < 3) &
#                         (df_cleaned['popularity_zscore'].abs() < 3)]
# rows_removed_step_2 = initial_row_count - len(df_cleaned)
# print(f'Rows removed after filtering extreme z-scores: {rows_removed_step_2}')

In [ ]:
# # Step 3: Remove rows with zero or negative revenue and budget
# initial_row_count = len(df_cleaned)
# df_cleaned = df_cleaned[(df_cleaned['revenue'] > 0) & (df_cleaned['budget'] > 0)]
# rows_removed_step_3 = initial_row_count - len(df_cleaned)
# print(f'Rows removed after filtering zero or negative revenue/budget: {rows_removed_step_3}')

In [ ]:
# remove rows with runtime under 40 minutes
initial_row_count = len(df_cleaned)
df_cleaned = df_cleaned[df_cleaned['runtime'] > 40]
rows_removed_step_4 = initial_row_count - len(df_cleaned)
print(f'Rows removed after filtering runtime under 40 minutes: {rows_removed_step_4}')

In [ ]:
# remove rows with low ratings (below a threshold of 1)
initial_row_count = len(df_cleaned)
df_cleaned = df_cleaned[df_cleaned['tmdb_rating'] > 1]
df_cleaned = df_cleaned[df_cleaned['imdb_rating'] > 1]
rows_removed_step_5 = initial_row_count - len(df_cleaned)
print(f'Rows removed after filtering low ratings: {rows_removed_step_5}')

In [ ]:
# remove duplicates
initial_row_count = len(df_cleaned)
df_cleaned = df_cleaned.drop_duplicates(subset=['tmdb_id', 'imdb_id', 'title'])
rows_removed_step_6 = initial_row_count - len(df_cleaned)
print(f'Rows removed after dropping duplicates: {rows_removed_step_6}')

In [ ]:
# remove rows with very few votes (below a threshold of 10)
initial_row_count = len(df_cleaned)
df_cleaned = df_cleaned[(df_cleaned['tmdb_votes'] >= 10) & (df_cleaned['imdb_votes'] >= 10)]
rows_removed_step_7 = initial_row_count - len(df_cleaned)
print(f'Rows removed after filtering low vote count: {rows_removed_step_7}')

In [ ]:
# cleaned dataset
final_row_count = len(df_cleaned)
print(f'Final number of rows in the cleaned dataset: {final_row_count}')

In [ ]:
df_cleaned.head()

### Data Overview

#### Descriptive Stats

In [ ]:
# Basic statistics for numerical columns
print("Descriptive Statistics:")
print(df_cleaned.describe())

# Get information on data types and non-null counts
print("\nData Types and Non-null Counts:")
print(df_cleaned.info())

# Check for unique values in categorical columns
print("\nUnique Values in Categorical Columns:")
print(df_cleaned[['genres', 'director', 'language', 'release_year']].nunique())


#### Check missing values

In [ ]:
# Check for missing values
print("\nMissing Values in Columns:")
print(df_cleaned.isnull().sum())


#### Correlation Matrix

In [ ]:
# Plot correlation matrix
corr = df_cleaned.corr()

# Heatmap of correlations
plt.figure(figsize=(12, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix')
plt.show()

#### Visualize Distributions

In [ ]:
# Histograms for numerical columns
df_cleaned[['budget', 'revenue', 'popularity', 'runtime', 'tmdb_rating', 'imdb_rating']].hist(figsize=(12, 10), bins=30)
plt.tight_layout()
plt.show()

# Boxplots for better understanding of the distributions
plt.figure(figsize=(12, 8))
sns.boxplot(data=df_cleaned[['budget', 'revenue', 'popularity', 'runtime', 'tmdb_rating', 'imdb_rating']])
plt.title('Boxplot of Numerical Columns')
plt.show()

#### Trends Over Time

In [ ]:
# Group data by release year and calculate mean values
yearly_trends = df_cleaned.groupby('release_year').agg({
    'budget': 'mean',
    'revenue': 'mean',
    'popularity': 'mean',
    'tmdb_rating': 'mean',
    'imdb_rating': 'mean',
    'runtime': 'mean'
}).reset_index()

# Plot trends over time
plt.figure(figsize=(12, 8))
plt.plot(yearly_trends['release_year'], yearly_trends['budget'], label='Average Budget', marker='o')
plt.plot(yearly_trends['release_year'], yearly_trends['revenue'], label='Average Revenue', marker='o')
plt.plot(yearly_trends['release_year'], yearly_trends['popularity'], label='Average Popularity', marker='o')
plt.plot(yearly_trends['release_year'], yearly_trends['tmdb_rating'], label='Average TMDB Rating', marker='o')
plt.plot(yearly_trends['release_year'], yearly_trends['imdb_rating'], label='Average IMDb Rating', marker='o')
plt.xlabel('Year')
plt.ylabel('Value')
plt.title('Trends Over Time (by Year)')
plt.legend()
plt.show()


#### Popularity and Revenue Relationship

In [ ]:
# Scatter plot for popularity vs revenue
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_cleaned, x='popularity', y='revenue', hue='release_year', palette='viridis')
plt.title('Popularity vs Revenue')
plt.xlabel('Popularity')
plt.ylabel('Revenue')
plt.show()

# Scatter plot for popularity vs budget
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_cleaned, x='popularity', y='budget', hue='release_year', palette='viridis')
plt.title('Popularity vs Budget')
plt.xlabel('Popularity')
plt.ylabel('Budget')
plt.show()


#### Rating Distribution by Genre

In [ ]:
# Take a 10% sample of the DataFrame for easier visualization
df_sample = df_cleaned.sample(frac=0.1, random_state=42)

# Split genres into separate rows for analysis
df_genre = df_sample[['genres', 'tmdb_rating', 'imdb_rating']].explode('genres')

# Plot TMDB Rating Distribution by Genre
plt.figure(figsize=(12, 8))
sns.boxplot(x='genres', y='tmdb_rating', data=df_genre)
plt.title('TMDB Rating Distribution by Genre (Sampled Data)')
plt.xticks(rotation=90)
plt.show()

# Plot IMDb Rating Distribution by Genre
plt.figure(figsize=(12, 8))
sns.boxplot(x='genres', y='imdb_rating', data=df_genre)
plt.title('IMDb Rating Distribution by Genre (Sampled Data)')
plt.xticks(rotation=90)
plt.show()

#### Analyze Directors

In [ ]:
# Group data by director and calculate the mean for key variables
director_stats = df_cleaned.groupby('director').agg({
    'budget': 'mean',
    'revenue': 'mean',
    'popularity': 'mean',
    'tmdb_rating': 'mean',
    'imdb_rating': 'mean',
    'runtime': 'mean'
}).reset_index()

# Top directors by average revenue
top_directors = director_stats.sort_values(by='revenue', ascending=False).head(10)
print("\nTop 10 Directors by Average Revenue:")
print(top_directors[['director', 'revenue']])

# Plot the top 10 directors by average revenue
plt.figure(figsize=(12, 8))
sns.barplot(x='revenue', y='director', data=top_directors)
plt.title('Top 10 Directors by Average Revenue')
plt.xlabel('Average Revenue')
plt.ylabel('Director')
plt.show()

#### Sentiment Analysis

In [ ]:
# Define a function to calculate sentiment polarity
def get_sentiment(text):
    return TextBlob(text).sentiment.polarity

# Apply the sentiment function to the 'title' or 'clean_title'
df_cleaned['sentiment'] = df_cleaned['title'].apply(get_sentiment)

# Plot sentiment distribution
plt.figure(figsize=(10, 6))
sns.histplot(df_cleaned['sentiment'], bins=30, kde=True)
plt.title('Sentiment Distribution')
plt.xlabel('Sentiment Polarity')
plt.ylabel('Frequency')
plt.show()

Votes vs Languages

In [ ]:
# Aggregate votes by language for both TMDB and IMDb
language_votes = df_cleaned.groupby('language')[['tmdb_votes', 'imdb_votes']].sum()

# Calculate the number of films per language and convert to DataFrame
language_counts = df_cleaned['language'].value_counts().to_frame(name='language_count')

# Merge the language_counts with the vote data
language_stats = language_votes.merge(language_counts, left_index=True, right_index=True)

# Calculate correlation between the votes and the number of films per language
correlation_tmdb = language_stats['tmdb_votes'].corr(language_stats['language_count'])
correlation_imdb = language_stats['imdb_votes'].corr(language_stats['language_count'])

# Print the correlations
print(f'Correlation between TMDB votes and number of films per language: {correlation_tmdb:.2f}')
print(f'Correlation between IMDb votes and number of films per language: {correlation_imdb:.2f}')


In [ ]:
# Visualize the distribution of TMDB votes by language (top 15 for readability)
top_languages_tmdb = language_stats['tmdb_votes'].sort_values(ascending=False).head(15)
top_languages_tmdb.plot(kind='bar', figsize=(12, 8), color='lightblue')
plt.title('Total TMDB Votes by Language (Top 15)')
plt.xlabel('Language')
plt.ylabel('Total TMDB Votes')
plt.xticks(rotation=90)
plt.show()

In [ ]:


# Visualize the distribution of IMDb votes by language (top 15 for readability)
top_languages_imdb = language_stats['imdb_votes'].sort_values(ascending=False).head(15)
top_languages_imdb.plot(kind='bar', figsize=(12, 8), color='salmon')
plt.title('Total IMDb Votes by Language (Top 15)')
plt.xlabel('Language')
plt.ylabel('Total IMDb Votes')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Scatter plot of TMDB votes vs. number of films per language
plt.figure(figsize=(10, 6))
sns.scatterplot(x=language_stats['language_count'], y=language_stats['tmdb_votes'])
plt.title('TMDB Votes vs. Number of Films per Language')
plt.xlabel('Number of Films per Language')
plt.ylabel('Total TMDB Votes')
plt.show()

In [ ]:
# Scatter plot of IMDb votes vs. number of films per language
plt.figure(figsize=(10, 6))
sns.scatterplot(x=language_stats['language_count'], y=language_stats['imdb_votes'])
plt.title('IMDb Votes vs. Number of Films per Language')
plt.xlabel('Number of Films per Language')
plt.ylabel('Total IMDb Votes')
plt.show()

Same without English

In [ ]:
# Exclude English language
df_non_english = df_cleaned[df_cleaned['language'] != 'English']

# Aggregate votes by language for both TMDB and IMDb (excluding English)
language_votes = df_non_english.groupby('language')[['tmdb_votes', 'imdb_votes']].sum()

# Calculate the number of films per language (excluding English) and convert to DataFrame
language_counts = df_non_english['language'].value_counts().to_frame(name='language_count')

# Merge the language_counts with the vote data
language_stats = language_votes.merge(language_counts, left_index=True, right_index=True)

# Calculate correlation between the votes and the number of films per language
correlation_tmdb = language_stats['tmdb_votes'].corr(language_stats['language_count'])
correlation_imdb = language_stats['imdb_votes'].corr(language_stats['language_count'])

# Print the correlations
print(f'Correlation between TMDB votes and number of films per language (excluding English): {correlation_tmdb:.2f}')
print(f'Correlation between IMDb votes and number of films per language (excluding English): {correlation_imdb:.2f}')

# Visualize the distribution of TMDB votes by language (top 15 for readability)
top_languages_tmdb = language_stats['tmdb_votes'].sort_values(ascending=False).head(15)
top_languages_tmdb.plot(kind='bar', figsize=(12, 8), color='lightblue')
plt.title('Total TMDB Votes by Language (Top 15) - Excluding English')
plt.xlabel('Language')
plt.ylabel('Total TMDB Votes')
plt.xticks(rotation=90)
plt.show()

# Visualize the distribution of IMDb votes by language (top 15 for readability)
top_languages_imdb = language_stats['imdb_votes'].sort_values(ascending=False).head(15)
top_languages_imdb.plot(kind='bar', figsize=(12, 8), color='salmon')
plt.title('Total IMDb Votes by Language (Top 15) - Excluding English')
